# Merging RTX and Robokop

This notebook summarizes our results when merging RTX and Robokop. using Translators Node Normalization.
For this we downloaded RTX KG2 v2.7.3 and Robo `c5ec1f282158182f`


## Summary

TODO


In [8]:
%%capture
# Import dependencies
import pyspark as ps
import os
from pathlib import Path
import subprocess
import pyspark.sql.functions as f
# import spark 
%load_ext autoreload
%autoreload 2
from rich.console import Console
from rich.logging import RichHandler
from rich.panel import Panel
console = Console()

# hack that moves this notebook context into the kedro path
root_path = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).decode().strip()
os.chdir(Path(root_path) / 'pipelines' / 'matrix')

# this loads various objects into the context, see 
# https://docs.kedro.org/en/stable/notebooks_and_ipython/kedro_and_notebooks.html#kedro-line-magics
%load_ext kedro.ipython
# %reload_kedro  --env cloud
# %reload_kedro  --env test
%reload_kedro


In [33]:
def top_20(df):
    return df._jdf.showString(20,200, False)

In [9]:
%%capture
unified_nodes = catalog.load("integration.prm.unified_nodes")
unified_edges = catalog.load("integration.prm.unified_edges")
robo_nodes = catalog.load("integration.int.robokop.nodes")
robo_edges = catalog.load("integration.int.robokop.edges")
rtx_nodes = catalog.load("integration.int.rtx.nodes")
rtx_edges = catalog.load("integration.int.rtx.edges")

24/10/07 17:52:05 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [20]:
console.rule("[bold blue]Unified KG")
console.print(Panel.fit(f"""
Unified Nodes: {unified_nodes.count()}
Robo Nodes: {robo_nodes.count()}
RTX Nodes: {rtx_nodes.count()}
""", title="Node Counts"))
# now edges
console.print(Panel.fit(f"""
Unified Edges: {unified_edges.count()}
Robo Edges: {robo_edges.count()}
RTX Edges: {rtx_edges.count()}
""", title="Edge Counts"))
# first calculate the number of nodes and edges in each kg
unified_node_count = unified_nodes.count()
unified_edge_count = unified_edges.count()
robo_node_count = robo_nodes.count()
robo_edge_count = robo_edges.count()
rtx_node_count = rtx_nodes.count()
rtx_edge_count = rtx_edges.count()

nodes_in_both = unified_nodes.filter(f.array_contains(f.col("upstream_data_source"), "rtxkg2") & f.array_contains(f.col("upstream_data_source"), "robokop"))
nodes_in_rtx = unified_nodes.filter(f.array_contains(f.col("upstream_data_source"), "rtxkg2"))
nodes_in_robo = unified_nodes.filter(f.array_contains(f.col("upstream_data_source"), "robokop"))

console.print(Panel.fit(
f"""
Nodes originating from RTX: {nodes_in_rtx.count()/unified_node_count*100:.2f}%
Nodes originating from Robo: {nodes_in_robo.count()/unified_node_count*100:.2f}%
Nodes originating from Both: {nodes_in_both.count()/unified_node_count*100:.2f}%
""", title="Node Origin Proportions"))

edges_in_both = unified_edges.filter(f.array_contains(f.col("upstream_data_source"), "rtxkg2") & f.array_contains(f.col("upstream_data_source"), "robokop"))
edges_in_rtx = unified_edges.filter(f.array_contains(f.col("upstream_data_source"), "rtxkg2"))
edges_in_robo = unified_edges.filter(f.array_contains(f.col("upstream_data_source"), "robokop"))

console.print(Panel.fit(
f"""
Edges originating from RTX: {edges_in_rtx.count()/unified_edge_count*100:.2f}%
Edges originating from Robo: {edges_in_robo.count()/unified_edge_count*100:.2f}%
Edges originating from Both: {edges_in_both.count()/unified_edge_count*100:.2f}%
""", title="Edge Origin Proportions"))


─────────────────────────────────────────────────── Unified KG ────────────────────────────────────────────────────

╭────── Node Counts ──────╮
│                         │
│ Unified Nodes: 12772268 │
│ Robo Nodes: 9748240     │
│ RTX Nodes: 3659686      │
│                         │
╰─────────────────────────╯

╭────── Edge Counts ───────╮
│                          │
│ Unified Edges: 150717205 │
│ Robo Edges: 137706046    │
│ RTX Edges: 18402956      │
│                          │
╰──────────────────────────╯

╭────── Node Origin Proportions ──────╮
│                                     │
│ Nodes originating from RTX: 28.55%  │
│ Nodes originating from Robo: 76.32% │
│ Nodes originating from Both: 4.88%  │
│                                     │
╰─────────────────────────────────────╯

╭────── Edge Origin Proportions ──────╮
│                                     │
│ Edges originating from RTX: 12.18%  │
│ Edges originating from Robo: 88.37% │
│ Edges originating from Both: 0.55%  │
│                                     │
╰─────────────────────────────────────╯

Wow that's not a lot of edges that are present in both. I wonder why there are so many more edges in Robokop as well. There's like 150M edges there and only 18M in RTX. 
Let's look at the predicate counts:


In [36]:
def stats_on_df(df: ps.sql.DataFrame, col: str, kg_name: str):
    df_counts = df.groupBy(col).count().sort("count", ascending=False)
    console.print(Panel.fit(top_20(df_counts), title=f"{col} Counts in {kg_name}"))

stats_on_df(edges_in_both, "predicate", "Both")
stats_on_df(edges_in_rtx, "predicate", "RTX")
stats_on_df(edges_in_robo, "predicate", "Robo")



╭─────────── predicate Counts in Both ────────────╮
│ +--------------------------------------+------+ │
│ |                             predicate| count| │
│ +--------------------------------------+------+ │
│ |                   biolink:subclass_of|490258| │
│ |               biolink:has_participant|243446| │
│ |                      biolink:has_part| 15244| │
│ |                    biolink:located_in| 13586| │
│ |                      biolink:in_taxon| 10876| │
│ |     biolink:physically_interacts_with| 10072| │
│ |                       biolink:affects|  9250| │
│ |                     biolink:has_input|  8539| │
│ |                    biolink:related_to|  8500| │
│ |                    biolink:has_output|  4834| │
│ |                        biolink:treats|  4782| │
│ |biolink:gene_associated_with_condition|  3733| │
│ |                 biolink:develops_from|  1548| │
│ |                 biolink:has_phenotype|  1534| │
│ |                 biolink:coexists_with|   827| │
│ |                        biolink:causes|   823| │
│ |                      biolink:overlaps|   681| │
│ |                  biolink:expressed_in|   573| │
│ |              biolink:colocalizes_with|    98| │
│ |                biolink:contributes_to|    94| │
│ +--------------------------------------+------+ │
│ only showing top 20 rows                        │
│                                                 │
╰─────────────────────────────────────────────────╯

╭──────────── predicate Counts in RTX ─────────────╮
│ +--------------------------------------+-------+ │
│ |                             predicate|  count| │
│ +--------------------------------------+-------+ │
│ |               biolink:has_participant|3837590| │
│ |                     biolink:occurs_in|3087968| │
│ |                   biolink:subclass_of|2983431| │
│ |     biolink:physically_interacts_with|2641479| │
│ |                    biolink:related_to|1535302| │
│ |biolink:gene_associated_with_condition| 839112| │
│ |                      biolink:in_taxon| 583731| │
│ |                      biolink:has_part| 403589| │
│ |                    biolink:has_output| 375980| │
│ |                     biolink:has_input| 364437| │
│ |              biolink:transcribed_from| 257221| │
│ |                    biolink:located_in| 239580| │
│ |                   biolink:close_match| 212699| │
│ |                biolink:interacts_with| 133377| │
│ |                       biolink:affects| 119041| │
│ |                 biolink:coexists_with|  97832| │
│ |               biolink:gene_product_of|  90416| │
│ |                       biolink:enables|  68829| │
│ |                        biolink:treats|  51621| │
│ |                 biolink:translates_to|  47331| │
│ +--------------------------------------+-------+ │
│ only showing top 20 rows                         │
│                                                  │
╰──────────────────────────────────────────────────╯

╭──────────────── predicate Counts in Robo ────────────────╮
│ +---------------------------------------------+--------+ │
│ |                                    predicate|   count| │
│ +---------------------------------------------+--------+ │
│ |                          biolink:subclass_of|60176118| │
│ |                 biolink:is_nearby_variant_of|31864140| │
│ |                              biolink:affects|18098919| │
│ |                     biolink:coexpressed_with| 9210882| │
│ |             biolink:is_non_coding_variant_of| 3858701| │
│ |            biolink:physically_interacts_with| 1415606| │
│ |                        biolink:has_phenotype| 1100999| │
│ |                         biolink:expressed_in| 1091384| │
│ |                 biolink:actively_involved_in|  952972| │
│ |   biolink:directly_physically_interacts_with|  815337| │
│ |                      biolink:has_participant|  558161| │
│ |                            biolink:regulates|  528959| │
│ |                                biolink:binds|  433800| │
│ |biolink:treats_or_applied_or_studied_to_treat|  324085| │
│ |                            biolink:occurs_in|  255150| │
│ |                            biolink:catalyzes|  237533| │
│ |                        biolink:homologous_to|  230627| │
│ |                           biolink:located_in|  222328| │
│ |                    biolink:has_adverse_event|  202144| │
│ |                             biolink:has_part|  191638| │
│ +---------------------------------------------+--------+ │
│ only showing top 20 rows                                 │
│                                                          │
╰──────────────────────────────────────────────────────────╯

In [37]:

stats_on_df(nodes_in_both, "category", "Both")
stats_on_df(nodes_in_rtx, "category", "RTX")
stats_on_df(nodes_in_robo, "category", "Robo")

╭─────────────── category Counts in Both ───────────────╮
│ +--------------------------------------------+------+ │
│ |                                    category| count| │
│ +--------------------------------------------+------+ │
│ |                             biolink:Protein|190001| │
│ |                     biolink:MolecularEntity|104015| │
│ |                       biolink:OrganismTaxon| 71901| │
│ |                       biolink:SmallMolecule| 67576| │
│ |                                biolink:Gene| 38703| │
│ |                             biolink:Disease| 35584| │
│ |                             biolink:Pathway| 31195| │
│ |          biolink:DiseaseOrPhenotypicFeature| 19509| │
│ |                    biolink:AnatomicalEntity| 15921| │
│ |                   biolink:MolecularActivity| 12931| │
│ |                      biolink:ChemicalEntity|  8977| │
│ |                          biolink:NamedThing|  5065| │
│ |   biolink:ChemicalEntityOrGeneOrGeneProduct|  4501| │
│ |          biolink:PhysicalEssenceOrOccurrent|  4031| │
│ |biolink:ChemicalEntityOrProteinOrPolypeptide|  2419| │
│ |                   biolink:PhenotypicFeature|  2032| │
│ |                                biolink:Cell|  1624| │
│ |                   biolink:CellularComponent|  1612| │
│ |                      biolink:ThingWithTaxon|  1609| │
│ |            biolink:GrossAnatomicalStructure|  1562| │
│ +--------------------------------------------+------+ │
│ only showing top 20 rows                              │
│                                                       │
╰───────────────────────────────────────────────────────╯

╭─────────── category Counts in RTX ───────────╮
│ +----------------------------------+-------+ │
│ |                          category|  count| │
│ +----------------------------------+-------+ │
│ |             biolink:OrganismTaxon|1785601| │
│ |                biolink:Transcript| 257221| │
│ |         biolink:MolecularActivity| 228645| │
│ |                   biolink:Protein| 197335| │
│ |            biolink:ChemicalEntity| 187144| │
│ |             biolink:SmallMolecule| 135790| │
│ |                      biolink:Drug| 132905| │
│ |           biolink:MolecularEntity| 130711| │
│ |          biolink:AnatomicalEntity| 124429| │
│ |                      biolink:Gene| 111966| │
│ |                   biolink:Disease|  84705| │
│ |                   biolink:Pathway|  69550| │
│ |               biolink:Polypeptide|  44705| │
│ |biolink:DiseaseOrPhenotypicFeature|  33237| │
│ |      biolink:PhysiologicalProcess|  33217| │
│ |          biolink:BiologicalEntity|  19908| │
│ |         biolink:NucleicAcidEntity|  15685| │
│ |  biolink:GrossAnatomicalStructure|  14866| │
│ |         biolink:CellularComponent|   6927| │
│ |                biolink:NamedThing|   5065| │
│ +----------------------------------+-------+ │
│ only showing top 20 rows                     │
│                                              │
╰──────────────────────────────────────────────╯

╭─────────────── category Counts in Robo ────────────────╮
│ +--------------------------------------------+-------+ │
│ |                                    category|  count| │
│ +--------------------------------------------+-------+ │
│ |                       biolink:GenomicEntity|5112649| │
│ |                          biolink:NamedThing|2471600| │
│ |          biolink:PhysicalEssenceOrOccurrent|1103707| │
│ |                     biolink:MolecularEntity| 319537| │
│ |                             biolink:Protein| 190001| │
│ |          biolink:MacromolecularMachineMixin|  98131| │
│ |                       biolink:OrganismTaxon|  71901| │
│ |                       biolink:SmallMolecule|  67576| │
│ |biolink:ChemicalEntityOrProteinOrPolypeptide|  55566| │
│ |                      biolink:ThingWithTaxon|  51483| │
│ |                                biolink:Gene|  38703| │
│ |                             biolink:Disease|  35584| │
│ |                             biolink:Pathway|  31195| │
│ |          biolink:DiseaseOrPhenotypicFeature|  27606| │
│ |   biolink:ChemicalEntityOrGeneOrGeneProduct|  23986| │
│ |                    biolink:AnatomicalEntity|  17938| │
│ |                   biolink:MolecularActivity|  12931| │
│ |                      biolink:ChemicalEntity|   8977| │
│ |                   biolink:PhenotypicFeature|   2032| │
│ |                                biolink:Cell|   1624| │
│ +--------------------------------------------+-------+ │
│ only showing top 20 rows                               │
│                                                        │
╰────────────────────────────────────────────────────────╯